In [2]:
import numpy as np
import pandas as pd
import configparser as cp
import timeit
import matplotlib.pyplot as plt
import pyfolio as pf
import ta
import configparser as cp

C:\Users\Jake Hanly\anaconda3\envs\mlfinlab\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
import os
os.environ['MLFINLAB_API_KEY'] = "ff083854904e114f02e7bc2a0553fece"
import mlfinlab as ml

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report, confusion_matrix, accuracy_score
from sklearn.utils import resample, shuffle
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression

In [5]:
import eikon as ek
ek.set_app_key('b3b6cfe33e0040d78b179a693e8af596df6a48c3')

In [ ]:
ek.__version__

In [ ]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')

In [6]:
Asset = ['ESc1']

2016-2019

In [7]:
open_data = ek.get_timeseries(Asset,
                        start_date='2016-01-01',
                        end_date='2019-10-29',
                        fields=('OPEN'),
                        interval='daily')

In [8]:
high_data = ek.get_timeseries(Asset,
                        start_date='2016-01-01',
                        end_date='2019-10-29',
                        fields=('HIGH'),
                        interval='daily')

In [9]:
low_data = ek.get_timeseries(Asset,
                        start_date='2016-01-01',
                        end_date='2019-10-29',
                        fields=('LOW'),
                        interval='daily')

In [10]:
close_data = ek.get_timeseries(Asset,
                        start_date='2016-01-01',
                        end_date='2019-10-29',
                        fields=('CLOSE'),
                        interval='daily')

In [11]:
volume_data = ek.get_timeseries(Asset,
                        start_date='2016-01-01',
                        end_date='2019-10-29',
                        fields=('VOLUME'),
                        interval='daily')


2019-current

In [12]:
open_data1 = ek.get_timeseries(Asset,
                        start_date='2019-10-30',
                        end_date='2023-03-15',
                        fields=('OPEN'),
                        interval='daily')

In [13]:
high_data1 = ek.get_timeseries(Asset,
                        start_date='2019-10-30',
                        end_date='2023-03-15',
                        fields=('HIGH'),
                        interval='daily')

In [14]:
low_data1 = ek.get_timeseries(Asset,
                        start_date='2019-10-30',
                        end_date='2023-03-15',
                        fields=('LOW'),
                        interval='daily')

In [15]:
close_data1 = ek.get_timeseries(Asset,
                        start_date='2019-10-30',
                        end_date='2023-03-15',
                        fields=('CLOSE'),
                        interval='daily')

In [16]:
volume_data1 = ek.get_timeseries(Asset,
                        start_date='2019-10-30',
                        end_date='2023-03-15',
                        fields=('VOLUME'),
                        interval='daily')


In [17]:
historical_data1 = pd.concat([open_data,high_data,low_data,close_data,volume_data], axis=1)
historical_data2 = pd.concat([open_data1,high_data1,low_data1,close_data1,volume_data1], axis=1)
historical_data = pd.concat([historical_data1, historical_data2], axis=0)

In [18]:
historical_data.fillna(0, inplace=True)

historical_data = historical_data.replace([np.inf, -np.inf], np.nan) # replace infinite values with NaN
for col in historical_data.columns:
    median = historical_data[col].median(skipna=True) # calculate the median of the non-NaN values in the column
    historical_data[col] = historical_data[col].fillna(median) # replace the NaN values with the median


In [19]:
historical_data.tail()

ESc1,OPEN,HIGH,LOW,CLOSE,VOLUME
Date,,,,,
2023-03-09,3997.00,4019.75,3909.50,3920.00,2547423
2023-03-10,3919.00,3941.75,3846.25,3862.75,3285811
2023-03-13,3887.75,3938.75,3808.75,3856.75,2750477
2023-03-14,3865.75,3938.00,3855.25,3920.50,1606509
2023-03-15,3920.50,3931.00,3834.00,3892.75,833643


get regression estimators

In [20]:
#MACD signal

# Calculate MACD indicator
ema_12 = historical_data['CLOSE'].ewm(span=12, adjust=False).mean()
ema_26 = historical_data['CLOSE'].ewm(span=26, adjust=False).mean()
macd = ema_12 - ema_26
signal = macd.ewm(span=9, adjust=False).mean()
histogram = macd - signal

historical_data['histogram'] = histogram

In [21]:
historical_data['side'] = np.nan

long_signals = historical_data['histogram'] >= 0 
short_signals = historical_data['histogram'] < 0
historical_data.loc[long_signals, 'side'] = 1
historical_data.loc[short_signals, 'side'] = -1


In [22]:
historical_data['log_ret'] = np.log(historical_data['CLOSE']).diff()

C:\Users\Jake Hanly\anaconda3\envs\mlfinlab\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [23]:
# Remove Look ahead biase by lagging the signal - for Base model
historical_data['side'] = historical_data['side'].shift(1)

In [24]:
historical_data.tail()

ESc1,OPEN,HIGH,LOW,CLOSE,VOLUME,histogram,side,log_ret
Date,,,,,,,,
2023-03-09,3997.00,4019.75,3909.50,3920.00,2547423,-11.265469,-1.0,-0.018952
2023-03-10,3919.00,3941.75,3846.25,3862.75,3285811,-16.734070,-1.0,-0.014712
2023-03-13,3887.75,3938.75,3808.75,3856.75,2750477,-19.549830,-1.0,-0.001555
2023-03-14,3865.75,3938.00,3855.25,3920.50,1606509,-16.059395,-1.0,0.016394
2023-03-15,3920.50,3931.00,3834.00,3892.75,833643,-14.602047,-1.0,-0.007103


In [25]:
#create model returns df
basemodeldata = historical_data.loc[:, ['CLOSE', 'histogram', 'log_ret', 'side']]

In [26]:
#calculate returns
basemodeldata['ret'] = basemodeldata['log_ret'] * basemodeldata['side']

In [27]:
#calculate bins
basemodeldata['bin'] = (basemodeldata['ret'] > 0).astype(int)

In [28]:
basemodeldata = basemodeldata.replace('#NAME?', np.nan)
basemodeldata = basemodeldata.replace([np.inf, -np.inf], np.nan)
basemodeldata = basemodeldata.fillna(0)

In [29]:
basemodeldata.tail()

ESc1,CLOSE,histogram,log_ret,side,ret,bin
Date,,,,,,
2023-03-09,3920.00,-11.265469,-0.018952,-1.0,0.018952,1
2023-03-10,3862.75,-16.734070,-0.014712,-1.0,0.014712,1
2023-03-13,3856.75,-19.549830,-0.001555,-1.0,0.001555,1
2023-03-14,3920.50,-16.059395,0.016394,-1.0,-0.016394,0
2023-03-15,3892.75,-14.602047,-0.007103,-1.0,0.007103,1


In [30]:
#lag the data before pairing the days return
basemodeldata = basemodeldata.shift(1)

In [31]:
#combine the data
historical_data = pd.concat([historical_data, basemodeldata[['ret']]], axis=1)

In [32]:
historical_data.tail()

ESc1,OPEN,HIGH,LOW,CLOSE,VOLUME,histogram,side,log_ret,ret
Date,,,,,,,,,
2023-03-09,3997.00,4019.75,3909.50,3920.00,2547423,-11.265469,-1.0,-0.018952,-0.001315
2023-03-10,3919.00,3941.75,3846.25,3862.75,3285811,-16.734070,-1.0,-0.014712,0.018952
2023-03-13,3887.75,3938.75,3808.75,3856.75,2750477,-19.549830,-1.0,-0.001555,0.014712
2023-03-14,3865.75,3938.00,3855.25,3920.50,1606509,-16.059395,-1.0,0.016394,0.001555
2023-03-15,3920.50,3931.00,3834.00,3892.75,833643,-14.602047,-1.0,-0.007103,-0.016394


In [33]:
# Import MlFinLab tools
from mlfinlab.labeling import trend_scanning_labels

# Loading data to use
ES_close = historical_data['CLOSE']

# Choosing a period where trends would be seen (mid-2008  -  mid-2009)
ES_close = ES_close.loc[pd.Timestamp(2017, 1, 5):pd.Timestamp(2023, 3, 15)]

# Getting indexes that we want to label
t_events = ES_close.index

# Fitting regressions to various windows up to 20 days back, using a minimum sample length of 5
tr_scan_labels = trend_scanning_labels(ES_close, t_events, observation_window=20,
                                       look_forward = False, min_sample_length=5, step=5)


<frozen mlfinlab.labeling.trend_scanning>:133: RuntimeWarning: divide by zero encountered in true_divide


In [34]:
historical_data['trend_tvalue'] = tr_scan_labels['t_value']
historical_data['tvalue_tm1'] = historical_data['trend_tvalue'].shift(1)

In [35]:
historical_data['trend'] = historical_data['tvalue_tm1'].rolling(window=20).mean()

In [36]:
raw_data = historical_data

In [37]:
raw_data = ta.add_all_ta_features(raw_data, "OPEN", "HIGH", "LOW", "CLOSE", "VOLUME", fillna=True)

C:\Users\Jake Hanly\anaconda3\envs\mlfinlab\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Jake Hanly\anaconda3\envs\mlfinlab\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Jake Hanly\anaconda3\envs\mlfinlab\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [38]:
raw_data.tail()

ESc1,OPEN,HIGH,LOW,CLOSE,VOLUME,histogram,side,log_ret,ret,trend_tvalue,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-09,3997.00,4019.75,3909.50,3920.00,2547423,-11.265469,-1.0,-0.018952,-0.001315,-5.058304,...,-0.496576,-0.216645,-0.279931,3.666112,1.985708,1.680404,4005.627981,-1.877347,-1.895193,95.121951
2023-03-10,3919.00,3941.75,3846.25,3862.75,3285811,-16.734070,-1.0,-0.014712,0.018952,-6.436892,...,-0.738713,-0.321058,-0.417655,8.520856,3.292738,5.228118,3995.621419,-1.460459,-1.471229,92.272275
2023-03-13,3887.75,3938.75,3808.75,3856.75,2750477,-19.549830,-1.0,-0.001555,0.014712,-6.269170,...,-0.933442,-0.443535,-0.489907,9.955162,4.625223,5.329940,3983.013022,-0.155330,-0.155451,91.973619
2023-03-14,3865.75,3938.00,3855.25,3920.50,1606509,-16.059395,-1.0,0.016394,0.001555,-5.844185,...,-0.947934,-0.544415,-0.403520,6.806060,5.061390,1.744670,3981.596378,1.652946,1.639434,95.146839
2023-03-15,3920.50,3931.00,3834.00,3892.75,833643,-14.602047,-1.0,-0.007103,-0.016394,-5.590101,...,-1.004729,-0.636478,-0.368251,1.127114,4.274535,-3.147421,3979.264488,-0.707818,-0.710335,93.765555


In [39]:
# Log Returns
#raw_data['log_ret'] = np.log(raw_data['CLOSE']).diff()

#PRICE DATA
# Momentum
raw_data['mom1'] = raw_data['CLOSE'].pct_change(periods=1)
raw_data['mom3'] = raw_data['CLOSE'].pct_change(periods=3)
raw_data['mom5'] = raw_data['CLOSE'].pct_change(periods=5)
raw_data['mom10'] = raw_data['CLOSE'].pct_change(periods=10)
raw_data['mom20'] = raw_data['CLOSE'].pct_change(periods=20)
raw_data['mom40'] = raw_data['CLOSE'].pct_change(periods=40)
raw_data['mom60'] = raw_data['CLOSE'].pct_change(periods=60)
raw_data['mom120'] = raw_data['CLOSE'].pct_change(periods=120)
raw_data['mom250'] = raw_data['CLOSE'].pct_change(periods=250)

# Volatility
raw_data['volatility_250'] = raw_data['log_ret'].rolling(window=250, min_periods=250, center=False).std()
raw_data['volatility_120'] = raw_data['log_ret'].rolling(window=120, min_periods=120, center=False).std()
raw_data['volatility_50'] = raw_data['log_ret'].rolling(window=50, min_periods=50, center=False).std()
raw_data['volatility_31'] = raw_data['log_ret'].rolling(window=31, min_periods=31, center=False).std()
raw_data['volatility_15'] = raw_data['log_ret'].rolling(window=15, min_periods=15, center=False).std()

# Serial Correlation (Takes about 4 minutes)
window_autocorr = 50

raw_data['autocorr_1'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
raw_data['autocorr_2'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=2), raw=False)
raw_data['autocorr_3'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=3), raw=False)
raw_data['autocorr_5'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=5), raw=False)
raw_data['autocorr_10'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=10), raw=False)

# Get the various log -t returns
raw_data['log_t1'] = raw_data['log_ret'].shift(1)
raw_data['log_t2'] = raw_data['log_ret'].shift(2)
raw_data['log_t3'] = raw_data['log_ret'].shift(3)
raw_data['log_t5'] = raw_data['log_ret'].shift(5)

#correlation



In [40]:
from mlfinlab.features.volatility_estimators import parkinson

o_h_l_c = raw_data.iloc[:,0:4]

parkinson_sinclair = parkinson(o_h_l_c, 22, False)

#add columnheader name
parkinson_sinclair.name = 'parkinson_sinclair'

print(parkinson_sinclair)

Date
2016-02-03    0.016161
2016-02-04    0.015799
2016-02-05    0.016063
2016-02-08    0.016409
2016-02-09    0.016110
                ...   
2023-03-09    0.010046
2023-03-10    0.010152
2023-03-13    0.010878
2023-03-14    0.010842
2023-03-15    0.011222
Name: parkinson_sinclair, Length: 1793, dtype: float64


In [41]:

raw_data = raw_data.join(parkinson_sinclair, how='outer')

In [42]:
# Calculating volatility with Parkinson estimator (De Prado formula)
parkinson_deprado = parkinson(o_h_l_c, 22, True)

#add columnheader name
parkinson_deprado.name = 'parkinson_deprado'

raw_data = raw_data.join(parkinson_deprado, how='outer')

In [43]:
from mlfinlab.features.volatility_estimators import garman_klass

# Calculating volatility with Garman-Klass estimator
garman_klass_volatility = garman_klass(o_h_l_c, window= 22)

#add columnheader name
garman_klass_volatility.name = 'garman_klass_volatility'

raw_data = raw_data.join(garman_klass_volatility, how='outer')

C:\Users\Jake Hanly\anaconda3\envs\mlfinlab\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [44]:
from mlfinlab.features.volatility_estimators import rogers_satchell

# Calculating volatility with Rogers-Satchell estimator
rogers_satchell_volatility = rogers_satchell(o_h_l_c, window= 20)

#add columnheader name
rogers_satchell_volatility.name = 'rogers_satchell_volatility'

raw_data = raw_data.join(rogers_satchell_volatility, how='outer')

In [45]:
from mlfinlab.features.volatility_estimators import CorwinShultzVolatility

# Calculating volatility with Corwin Shultz estimator
corwin_shultz_temp = CorwinShultzVolatility(o_h_l_c ,22)
corwin_shultz_volatility = corwin_shultz_temp.calculate_volatility()

#add columnheader name
corwin_shultz_volatility.name = 'corwin_shultz_volatility'

raw_data = raw_data.join(corwin_shultz_volatility, how='outer')


In [46]:

from mlfinlab.features.volatility_estimators import cho_frees


# Calculating volatility with Cho Frees estimator
cho_frees_volatility = cho_frees(o_h_l_c, 0.01)

#add columnheader name
cho_frees_volatility.name = 'cho_frees_volatility'

raw_data = raw_data.join(cho_frees_volatility, how='outer')

In [47]:
from mlfinlab.features.volatility_estimators import first_exit_times

# Calculating volatility with first exit times estimator
first_exit_times_volatility = first_exit_times(o_h_l_c, 0.01, 22)

#add columnheader name
first_exit_times_volatility.name = 'first_exit_times_volatility'

raw_data = raw_data.join(first_exit_times_volatility, how='outer')

In [48]:
raw_data.fillna(0, inplace=True)

In [49]:
print(raw_data)

               OPEN     HIGH      LOW    CLOSE   VOLUME  histogram  side  \
Date                                                                       
2016-01-04  2037.75  2043.50  1980.25  2009.00  2216549   0.000000   0.0   
2016-01-04  2037.75  2043.50  1980.25  2009.00  2216549   0.000000   0.0   
2016-01-05  2009.00  2017.00  1992.25  2011.75  1688974   0.175499   1.0   
2016-01-06  2010.75  2013.25  1970.50  1986.00  2237518  -1.367407   1.0   
2016-01-07  1987.75  1991.00  1928.50  1933.00  2969295  -5.642408  -1.0   
...             ...      ...      ...      ...      ...        ...   ...   
2023-03-09  3997.00  4019.75  3909.50  3920.00  2547423 -11.265469  -1.0   
2023-03-10  3919.00  3941.75  3846.25  3862.75  3285811 -16.734070  -1.0   
2023-03-13  3887.75  3938.75  3808.75  3856.75  2750477 -19.549830  -1.0   
2023-03-14  3865.75  3938.00  3855.25  3920.50  1606509 -16.059395  -1.0   
2023-03-15  3920.50  3931.00  3834.00  3892.75   833643 -14.602047  -1.0   

           

In [50]:
rics = [
    'ESc1',
    'TYc1',
    'VXc1',
    'NQc1',
    'HYc1',
    '.DXY',
    'GCc1',
    'CLc1',
    '.IBXXIBHY',
    '.STOXX50E',
    'XLP',
    'XLY',
    '.VIX',
    'VBR',
    'GSLC.K',
    'SDAZ3',
    'SDAZ24',
    'FFcm1',
    '.IBXXIBIG',
    '.SKEWX',
    '.BCOM',
    '.RXM',
    '.MOVE',
    '.VVIX',
    '.VSTG',
    '.SPRP10GT',
    '.SPSGMITR',
    '.SPRPRVP',
    '.SPECFRLS',
    '.SPRPRCP',
    '.SPECFRP',
    '.SPECFR6P',
    '.SPXDPU1',
    '.RTOE.U',
    '.NEIXCTA',
    '.dMIUS0000PP',
    '.dMIUS0000JP',
    '.dMIUS0000OPUS',
    '.dMIUS0000IPUS',
    '.dMIUS0000NPUS',
    '.EHI474',
    '.TRIFTUSPLUT',
    '.FTFTUSPSUT',
    '.FTJQUA',
    '.FTJVAL',
    '.FTJMOM',
    '.FTJMIN',
    '.SPHYDA',
    '.NQBUY',
    '.SGIXSENT',
    '.VIX3M',
    '.WTEPS',
    'VQLc1',
    '.FTSE',
    'IDEV.K',
    '.HSI',
    'EEM'
]



In [51]:
Ricsdata2017 = ek.get_timeseries(rics,
                                start_date='2017-01-01',
                                end_date='2017-03-01',
                                fields='CLOSE',
                                interval='daily')
Ricsdata2017.head()

2023-04-23 18:57:47,323 P[21888] [MainThread 20508] Error with SDAZ3: No data available for the requested date range
2023-04-23 18:57:47,324 P[21888] [MainThread 20508] Error with SDAZ24: No data available for the requested date range
2023-04-23 18:57:47,325 P[21888] [MainThread 20508] Error with .TRIFTUSPLUT: No data available for the requested date range
2023-04-23 18:57:47,325 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:47,326 P[21888] [MainThread 20508] Error with .FTJQUA: No data available for the requested date range
2023-04-23 18:57:47,327 P[21888] [MainThread 20508] Error with .FTJVAL: No data available for the requested date range
2023-04-23 18:57:47,327 P[21888] [MainThread 20508] Error with .FTJMOM: No data available for the requested date range
2023-04-23 18:57:47,328 P[21888] [MainThread 20508] Error with .FTJMIN: No data available for the requested date range
2023-04-23 18:57:47,328 P[21888] [MainThre

CLOSE,ESc1,TYc1,VXc1,NQc1,HYc1,.DXY,GCc1,CLc1,.IBXXIBHY,.STOXX50E,...,.dMIUS0000NPUS,.EHI474,.SPHYDA,.NQBUY,.SGIXSENT,.VIX3M,.WTEPS,.FTSE,.HSI,EEM
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-02,NaN,NaN,NaN,NaN,NaN,102.83,NaN,NaN,NaN,3308.67,...,2649.302,NaN,NaN,NaN,NaN,NaN,313.65,NaN,NaN,NaN
2017-01-03,2252.50,124.109375,13.775,4906.25,87.42,103.21,1160.4,52.33,116.296256,3315.02,...,2661.110,NaN,2822.55,993.70,288.86,15.56,316.33,7177.89,22150.40,35.217562
2017-01-04,2264.25,124.109375,13.175,4933.50,88.15,102.70,1163.8,53.26,116.715608,3317.52,...,2675.904,NaN,2847.71,1003.81,289.55,14.84,318.57,7189.74,22134.47,35.485943
2017-01-05,2264.25,124.828125,13.025,4962.00,87.15,101.52,1179.7,53.76,116.989973,3316.47,...,2674.593,NaN,2838.01,1005.25,290.09,14.70,317.65,7195.31,22456.69,35.873604
2017-01-06,2271.50,124.406250,12.875,5004.00,88.23,102.22,1171.9,53.99,117.059031,3321.17,...,2681.643,NaN,2839.72,1007.13,290.30,14.56,318.33,7210.05,22503.01,35.724504


In [52]:
Ricsdata2017_1 = ek.get_timeseries(rics,
                                start_date='2017-04-02',
                                end_date='2017-6-01',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:47,887 P[21888] [MainThread 20508] Error with SDAZ3: No data available for the requested date range
2023-04-23 18:57:47,888 P[21888] [MainThread 20508] Error with SDAZ24: No data available for the requested date range
2023-04-23 18:57:47,888 P[21888] [MainThread 20508] Error with .TRIFTUSPLUT: No data available for the requested date range
2023-04-23 18:57:47,889 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:47,889 P[21888] [MainThread 20508] Error with .FTJQUA: No data available for the requested date range
2023-04-23 18:57:47,890 P[21888] [MainThread 20508] Error with .FTJVAL: No data available for the requested date range
2023-04-23 18:57:47,890 P[21888] [MainThread 20508] Error with .FTJMOM: No data available for the requested date range
2023-04-23 18:57:47,892 P[21888] [MainThread 20508] Error with .FTJMIN: No data available for the requested date range
2023-04-23 18:57:47,893 P[21888] [MainThre

In [53]:
Ricsdata2017_2 = ek.get_timeseries(rics,
                                start_date='2017-06-02',
                                end_date='2017-08-01',
                                fields='CLOSE',
                                interval='daily')


2023-04-23 18:57:48,410 P[21888] [MainThread 20508] Error with SDAZ3: No data available for the requested date range
2023-04-23 18:57:48,410 P[21888] [MainThread 20508] Error with SDAZ24: No data available for the requested date range
2023-04-23 18:57:48,411 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:48,412 P[21888] [MainThread 20508] Error with .FTJQUA: No data available for the requested date range
2023-04-23 18:57:48,413 P[21888] [MainThread 20508] Error with .FTJVAL: No data available for the requested date range
2023-04-23 18:57:48,414 P[21888] [MainThread 20508] Error with .FTJMOM: No data available for the requested date range
2023-04-23 18:57:48,414 P[21888] [MainThread 20508] Error with .FTJMIN: No data available for the requested date range
2023-04-23 18:57:48,415 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [54]:
Ricsdata2017_3 = ek.get_timeseries(rics,
                                start_date='2017-08-02',
                                end_date='2017-09-30',
                                fields='CLOSE',
                                interval='daily')


2023-04-23 18:57:48,864 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:48,864 P[21888] [MainThread 20508] Error with .FTJQUA: No data available for the requested date range
2023-04-23 18:57:48,865 P[21888] [MainThread 20508] Error with .FTJVAL: No data available for the requested date range
2023-04-23 18:57:48,866 P[21888] [MainThread 20508] Error with .FTJMOM: No data available for the requested date range
2023-04-23 18:57:48,867 P[21888] [MainThread 20508] Error with .FTJMIN: No data available for the requested date range
2023-04-23 18:57:48,867 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [55]:
Ricsdata2017_4 = ek.get_timeseries(rics,
                                start_date='2017-10-01',
                                end_date='2017-11-30',
                                fields='CLOSE',
                                interval='daily')


2023-04-23 18:57:49,427 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:49,428 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [56]:
Ricsdata2017_5 = ek.get_timeseries(rics,
                                start_date='2017-10-01',
                                end_date='2017-12-31',
                                fields='CLOSE',
                                interval='daily')


2023-04-23 18:57:50,060 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:50,061 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [57]:
Ricsdata2018 = ek.get_timeseries(rics,
                                start_date='2018-01-01',
                                end_date='2018-03-01',
                                fields='CLOSE',
                                interval='daily')


2023-04-23 18:57:50,450 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:50,451 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [58]:
Ricsdata2018_1 = ek.get_timeseries(rics,
                                start_date='2018-04-02',
                                end_date='2018-6-01',
                                fields='CLOSE',
                                interval='daily')


2023-04-23 18:57:50,840 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:50,841 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [59]:
Ricsdata2018_2 = ek.get_timeseries(rics,
                                start_date='2018-06-02',
                                end_date='2018-08-01',
                                fields='CLOSE',
                                interval='daily')


2023-04-23 18:57:51,230 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:51,231 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [60]:
Ricsdata2018_3 = ek.get_timeseries(rics,
                                start_date='2018-08-02',
                                end_date='2018-09-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:51,622 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:51,622 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [61]:
Ricsdata2018_4 = ek.get_timeseries(rics,
                                start_date='2018-10-01',
                                end_date='2018-11-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:52,008 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:52,008 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [62]:
Ricsdata2018_5 = ek.get_timeseries(rics,
                                start_date='2018-10-01',
                                end_date='2018-12-31',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:52,401 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:52,402 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [63]:
Ricsdata2019 = ek.get_timeseries(rics,
                                start_date='2019-01-01',
                                end_date='2019-03-01',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:52,954 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:52,955 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [64]:
Ricsdata2019_1 = ek.get_timeseries(rics,
                                start_date='2019-04-02',
                                end_date='2019-6-01',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:53,293 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:53,294 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [65]:
Ricsdata2019_2 = ek.get_timeseries(rics,
                                start_date='2019-06-02',
                                end_date='2019-08-01',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:53,637 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:53,638 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [66]:
Ricsdata2019_3 = ek.get_timeseries(rics,
                                start_date='2019-08-02',
                                end_date='2019-09-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:53,973 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:53,974 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [67]:
Ricsdata2019_4 = ek.get_timeseries(rics,
                                start_date='2019-10-01',
                                end_date='2019-11-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:54,279 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:54,279 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [68]:
Ricsdata2019_5 = ek.get_timeseries(rics,
                                start_date='2019-10-01',
                                end_date='2019-12-31',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:54,643 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:54,645 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [69]:
Ricsdata2020_1 = ek.get_timeseries(rics,
                                start_date='2020-01-01',
                                end_date='2020-02-28',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:55,033 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:55,034 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [70]:
Ricsdata2020_2 = ek.get_timeseries(rics,
                                start_date='2020-03-01',
                                end_date='2020-04-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:55,378 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:55,378 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [71]:
Ricsdata2020_3 = ek.get_timeseries(rics,
                                start_date='2020-05-01',
                                end_date='2020-06-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:55,812 P[21888] [MainThread 20508] Error with .SPECFRLS: No data available for the requested date range
2023-04-23 18:57:55,813 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:55,813 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [72]:
Ricsdata2020_4 = ek.get_timeseries(rics,
                                start_date='2020-07-01',
                                end_date='2020-08-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:56,235 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range
2023-04-23 18:57:56,235 P[21888] [MainThread 20508] Error with VQLc1: No data available for the requested date range


In [73]:
Ricsdata2020_5 = ek.get_timeseries(rics,
                                start_date='2020-09-01',
                                end_date='2020-10-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:56,631 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range


In [74]:
Ricsdata2020_6 = ek.get_timeseries(rics,
                                start_date='2020-11-01',
                                end_date='2020-12-31',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:57:57,201 P[21888] [MainThread 20508] Error with .FTFTUSPSUT: No data available for the requested date range


In [75]:
Ricsdata2021 = ek.get_timeseries(rics,
                                start_date='2021-01-01',
                                end_date='2021-02-28',
                                fields='CLOSE',
                                interval='daily')

In [76]:
Ricsdata2021_1 = ek.get_timeseries(rics,
                                start_date='2021-03-01',
                                end_date='2021-04-30',
                                fields='CLOSE',
                                interval='daily')

In [77]:
Ricsdata2021_2 = ek.get_timeseries(rics,
                                start_date='2021-05-01',
                                end_date='2021-06-30',
                                fields='CLOSE',
                                interval='daily')

In [78]:
Ricsdata2021_3 = ek.get_timeseries(rics,
                                start_date='2021-07-01',
                                end_date='2021-08-31',
                                fields='CLOSE',
                                interval='daily')

In [79]:
Ricsdata2021_4 = ek.get_timeseries(rics,
                                start_date='2021-09-01',
                                end_date='2021-10-30',
                                fields='CLOSE',
                                interval='daily')

In [80]:
Ricsdata2021_5 = ek.get_timeseries(rics,
                                start_date='2021-11-01',
                                end_date='2021-12-31',
                                fields='CLOSE',
                                interval='daily')

In [81]:
Ricsdata2022 = ek.get_timeseries(rics,
                                start_date='2022-01-01',
                                end_date='2022-02-28',
                                fields='CLOSE',
                                interval='daily')

In [82]:
Ricsdata2022_1 = ek.get_timeseries(rics,
                                start_date='2022-03-01',
                                end_date='2022-04-30',
                                fields='CLOSE',
                                interval='daily')

In [83]:
Ricsdata2022_2 = ek.get_timeseries(rics,
                                start_date='2022-05-01',
                                end_date='2022-06-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:58:00,764 P[21888] [MainThread 20508] Error with .NQBUY: No data available for the requested date range


In [84]:
Ricsdata2022_3 = ek.get_timeseries(rics,
                                start_date='2022-07-01',
                                end_date='2022-08-31',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:58:01,184 P[21888] [MainThread 20508] Error with .NQBUY: No data available for the requested date range


In [85]:
Ricsdata2022_4 = ek.get_timeseries(rics,
                                start_date='2022-09-01',
                                end_date='2022-10-30',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:58:01,544 P[21888] [MainThread 20508] Error with .NQBUY: No data available for the requested date range


In [86]:
Ricsdata2022_5 = ek.get_timeseries(rics,
                                start_date='2022-11-01',
                                end_date='2022-12-31',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:58:01,946 P[21888] [MainThread 20508] Error with .NQBUY: No data available for the requested date range


In [87]:
Ricsdata2023 = ek.get_timeseries(rics,
                                start_date='2023-01-01',
                                end_date='2023-03-16',
                                fields='CLOSE',
                                interval='daily')

2023-04-23 18:58:02,338 P[21888] [MainThread 20508] Error with .NQBUY: No data available for the requested date range


In [88]:
macrodata = pd.concat([Ricsdata2017, Ricsdata2017_1, Ricsdata2017_2, Ricsdata2017_3, Ricsdata2017_4, Ricsdata2017_5, Ricsdata2018, Ricsdata2018_1, Ricsdata2018_2, Ricsdata2018_3, Ricsdata2018_4, Ricsdata2018_5, Ricsdata2019, Ricsdata2020_1, Ricsdata2020_2, Ricsdata2020_3, Ricsdata2020_4, Ricsdata2020_5, Ricsdata2021, Ricsdata2021_1, Ricsdata2021_2, Ricsdata2021_3, Ricsdata2021_4, Ricsdata2021_5, Ricsdata2022, Ricsdata2022_1, Ricsdata2022_2, Ricsdata2022_3, Ricsdata2022_4, Ricsdata2022_5, Ricsdata2023])

In [89]:
macrodata.head()

,ESc1,TYc1,VXc1,NQc1,HYc1,.DXY,GCc1,CLc1,.IBXXIBHY,.STOXX50E,...,IDEV.K,.TRIFTUSPLUT,SDAZ3,SDAZ24,.FTJQUA,.FTJVAL,.FTJMOM,.FTJMIN,VQLc1,.FTFTUSPSUT
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-02,NaN,NaN,NaN,NaN,NaN,102.83,NaN,NaN,NaN,3308.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,2252.50,124.109375,13.775,4906.25,87.42,103.21,1160.4,52.33,116.296256,3315.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,2264.25,124.109375,13.175,4933.50,88.15,102.70,1163.8,53.26,116.715608,3317.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,2264.25,124.828125,13.025,4962.00,87.15,101.52,1179.7,53.76,116.989973,3316.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-06,2271.50,124.406250,12.875,5004.00,88.23,102.22,1171.9,53.99,117.059031,3321.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
import pandas as pd
from fredapi import Fred
from datetime import datetime, timedelta

In [91]:
# API key
fred = Fred(api_key='f49cd11c66241b63f7e1f146990465ab')

In [92]:

# Start and end dates for the data
start_date = input("2020-01-01")
end_date = input("2023-03-15")

# API key for FRED
api_key = 'f49cd11c66241b63f7e1f146990465ab'

# List of FRED codes to pull data for
fred_codes = ['DGS1', 'DGS2', 'DGS3', 'DGS5', 'DGS10', 'DGS30', 'WGS3MO', 'DTWEXBGS', 'RTWEXBGS',
              'T10Y3M', 'T10Y2Y', 'BAMLH0A0HYM2', 'T5YIFR', 'T5YIE', 'DFEDTARU', 'DFF', 'OBMMIJUMBO30YF',
              'T10Y2Y', 'FEDFUNDS', 'WALCL', 'BAMLH0A0HYM2', 'CPIAUCSL', 'M2SL', 'T10Y3M', 'UNRATE',
              'DGS10', 'CSUSHPINSA', 'RRPONTSYD', 'GDP', 'GDPC1', 'T5YIE', 'PSAVERT', 'ICSA', 'DEXUSUK' ]

# Create empty DataFrame to hold the data
fred_data = pd.DataFrame()

# Loop through each FRED code and pull the data
for code in fred_codes:
    try:
        data = Fred(api_key=api_key).get_series(code, start_date=start_date, end_date=end_date)
        if data.index.freq != 'D':
            data = data.resample('D').last().ffill()
        fred_data[code] = data
    except Exception as e:
        print(f'Error pulling data for {code}: {e}')

# Display the resulting DataFrame
print(fred_data.head())


2020-01-01 
2023-03-15 


            DGS1  DGS2  DGS3  DGS5  DGS10  DGS30  WGS3MO  DTWEXBGS  RTWEXBGS  \
1962-01-02  3.22   NaN  3.70  3.88   4.06    NaN     NaN       NaN       NaN   
1962-01-03  3.24   NaN  3.70  3.87   4.03    NaN     NaN       NaN       NaN   
1962-01-04  3.24   NaN  3.69  3.86   3.99    NaN     NaN       NaN       NaN   
1962-01-05  3.26   NaN  3.71  3.89   4.02    NaN     NaN       NaN       NaN   
1962-01-06  3.26   NaN  3.71  3.89   4.02    NaN     NaN       NaN       NaN   

            T10Y3M  ...  CPIAUCSL   M2SL  UNRATE  CSUSHPINSA  RRPONTSYD  \
1962-01-02     NaN  ...     30.04  337.5     5.8         NaN        NaN   
1962-01-03     NaN  ...     30.04  337.5     5.8         NaN        NaN   
1962-01-04     NaN  ...     30.04  337.5     5.8         NaN        NaN   
1962-01-05     NaN  ...     30.04  337.5     5.8         NaN        NaN   
1962-01-06     NaN  ...     30.04  337.5     5.8         NaN        NaN   

                GDP     GDPC1  PSAVERT  ICSA  DEXUSUK  
1962-01-02  

In [93]:
fred_data.head()

,DGS1,DGS2,DGS3,DGS5,DGS10,DGS30,WGS3MO,DTWEXBGS,RTWEXBGS,T10Y3M,...,CPIAUCSL,M2SL,UNRATE,CSUSHPINSA,RRPONTSYD,GDP,GDPC1,PSAVERT,ICSA,DEXUSUK
1962-01-02,3.22,NaN,3.70,3.88,4.06,NaN,NaN,NaN,NaN,NaN,...,30.04,337.5,5.8,NaN,NaN,594.013,3502.298,11.3,NaN,NaN
1962-01-03,3.24,NaN,3.70,3.87,4.03,NaN,NaN,NaN,NaN,NaN,...,30.04,337.5,5.8,NaN,NaN,594.013,3502.298,11.3,NaN,NaN
1962-01-04,3.24,NaN,3.69,3.86,3.99,NaN,NaN,NaN,NaN,NaN,...,30.04,337.5,5.8,NaN,NaN,594.013,3502.298,11.3,NaN,NaN
1962-01-05,3.26,NaN,3.71,3.89,4.02,NaN,NaN,NaN,NaN,NaN,...,30.04,337.5,5.8,NaN,NaN,594.013,3502.298,11.3,NaN,NaN
1962-01-06,3.26,NaN,3.71,3.89,4.02,NaN,NaN,NaN,NaN,NaN,...,30.04,337.5,5.8,NaN,NaN,594.013,3502.298,11.3,NaN,NaN


In [94]:
# Forward-fill the missing values for monthly data
fred_data = fred_data.fillna(method='ffill')

In [95]:
# Resample the data to daily frequency
fred_data = fred_data.resample('D').ffill()

# Display the resulting DataFrame
print(fred_data.tail())

            DGS1  DGS2  DGS3  DGS5  DGS10  DGS30  WGS3MO  DTWEXBGS  RTWEXBGS  \
2023-04-16  4.77  4.08  3.83  3.60   3.52   3.74    5.08  119.0301  115.5583   
2023-04-17  4.80  4.18  3.92  3.69   3.60   3.81    5.08  119.0301  115.5583   
2023-04-18  4.81  4.19  3.92  3.69   3.58   3.79    5.08  119.0301  115.5583   
2023-04-19  4.84  4.24  3.97  3.71   3.60   3.79    5.08  119.0301  115.5583   
2023-04-20  4.77  4.14  3.87  3.63   3.54   3.75    5.08  119.0301  115.5583   

            T10Y3M  ...  CPIAUCSL     M2SL  UNRATE  CSUSHPINSA  RRPONTSYD  \
2023-04-16   -1.62  ...   301.808  21062.5     3.5     292.706   2253.786   
2023-04-17   -1.61  ...   301.808  21062.5     3.5     292.706   2256.845   
2023-04-18   -1.62  ...   301.808  21062.5     3.5     292.706   2238.994   
2023-04-19   -1.56  ...   301.808  21062.5     3.5     292.706   2294.677   
2023-04-20   -1.58  ...   301.808  21062.5     3.5     292.706   2277.259   

                  GDP      GDPC1  PSAVERT      ICSA  DEX

In [96]:
# Calculate the percent change from the last period
fred_pct_change = fred_data.pct_change()

# Display the resulting DataFrame
print(fred_pct_change.tail())

                DGS1      DGS2      DGS3      DGS5     DGS10     DGS30  \
2023-04-16  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2023-04-17  0.006289  0.024510  0.023499  0.025000  0.022727  0.018717   
2023-04-18  0.002083  0.002392  0.000000  0.000000 -0.005556 -0.005249   
2023-04-19  0.006237  0.011933  0.012755  0.005420  0.005587  0.000000   
2023-04-20 -0.014463 -0.023585 -0.025189 -0.021563 -0.016667 -0.010554   

            WGS3MO  DTWEXBGS  RTWEXBGS    T10Y3M  ...  CPIAUCSL  M2SL  UNRATE  \
2023-04-16     0.0       0.0       0.0  0.000000  ...       0.0   0.0     0.0   
2023-04-17     0.0       0.0       0.0 -0.006173  ...       0.0   0.0     0.0   
2023-04-18     0.0       0.0       0.0  0.006211  ...       0.0   0.0     0.0   
2023-04-19     0.0       0.0       0.0 -0.037037  ...       0.0   0.0     0.0   
2023-04-20     0.0       0.0       0.0  0.012821  ...       0.0   0.0     0.0   

            CSUSHPINSA  RRPONTSYD  GDP  GDPC1  PSAVERT  ICSA  DEXUSU

In [97]:
# convert index to datetime format
fred_pct_change.index = pd.to_datetime(fred_pct_change.index)

# select rows after a certain date
start_date = '2016-01-01'
end_date = '2023-03-16'
mask = (fred_pct_change.index >= start_date) & (fred_pct_change.index <= end_date)
new_df = fred_pct_change.loc[mask]

print(new_df)

                DGS1      DGS2      DGS3      DGS5     DGS10     DGS30  \
2016-01-01  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2016-01-02  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2016-01-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2016-01-04 -0.061538 -0.037736  0.000000 -0.017045 -0.013216 -0.009967   
2016-01-05  0.114754  0.019608  0.007634  0.000000  0.004464  0.010067   
...              ...       ...       ...       ...       ...       ...   
2023-03-12  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2023-03-13 -0.122449 -0.123913 -0.099768 -0.070707 -0.040541  0.000000   
2023-03-14  0.034884  0.042184  0.043814  0.027174  0.025352  0.018919   
2023-03-15 -0.058427 -0.064286 -0.054321 -0.050265 -0.035714 -0.018568   
2023-03-16  0.071599  0.053435  0.041775  0.036212  0.014245  0.002703   

            WGS3MO  DTWEXBGS  RTWEXBGS    T10Y3M  ...  CPIAUCSL     M2SL  \
2016-01-01     0.0  0.000000  0.021

In [98]:
# Merge the two dataframes based on the 'Date' column
final_data = pd.merge(raw_data, macrodata, left_index=True, right_index=True, how='outer')
final_data = final_data.merge(new_df, left_index=True, right_index=True, how='inner')



In [99]:
print(final_data)

               OPEN     HIGH      LOW    CLOSE   VOLUME  histogram  side  \
2016-01-04  2037.75  2043.50  1980.25  2009.00  2216549   0.000000   0.0   
2016-01-04  2037.75  2043.50  1980.25  2009.00  2216549   0.000000   0.0   
2016-01-05  2009.00  2017.00  1992.25  2011.75  1688974   0.175499   1.0   
2016-01-06  2010.75  2013.25  1970.50  1986.00  2237518  -1.367407   1.0   
2016-01-07  1987.75  1991.00  1928.50  1933.00  2969295  -5.642408  -1.0   
...             ...      ...      ...      ...      ...        ...   ...   
2023-03-10  3919.00  3941.75  3846.25  3862.75  3285811 -16.734070  -1.0   
2023-03-13  3887.75  3938.75  3808.75  3856.75  2750477 -19.549830  -1.0   
2023-03-14  3865.75  3938.00  3855.25  3920.50  1606509 -16.059395  -1.0   
2023-03-15  3920.50  3931.00  3834.00  3892.75   833643 -14.602047  -1.0   
2023-03-16      NaN      NaN      NaN      NaN     <NA>        NaN   NaN   

             log_ret       ret  trend_tvalue  ...  CPIAUCSL  M2SL  UNRATE  \
2016-01-04

In [100]:
final_data.columns.to_list()

['OPEN',
 'HIGH',
 'LOW',
 'CLOSE',
 'VOLUME',
 'histogram',
 'side',
 'log_ret',
 'ret',
 'trend_tvalue',
 'tvalue_tm1',
 'trend',
 'volume_adi',
 'volume_obv',
 'volume_cmf',
 'volume_fi',
 'volume_em',
 'volume_sma_em',
 'volume_vpt',
 'volume_vwap',
 'volume_mfi',
 'volume_nvi',
 'volatility_bbm',
 'volatility_bbh',
 'volatility_bbl',
 'volatility_bbw',
 'volatility_bbp',
 'volatility_bbhi',
 'volatility_bbli',
 'volatility_kcc',
 'volatility_kch',
 'volatility_kcl',
 'volatility_kcw',
 'volatility_kcp',
 'volatility_kchi',
 'volatility_kcli',
 'volatility_dcl',
 'volatility_dch',
 'volatility_dcm',
 'volatility_dcw',
 'volatility_dcp',
 'volatility_atr',
 'volatility_ui',
 'trend_macd',
 'trend_macd_signal',
 'trend_macd_diff',
 'trend_sma_fast',
 'trend_sma_slow',
 'trend_ema_fast',
 'trend_ema_slow',
 'trend_vortex_ind_pos',
 'trend_vortex_ind_neg',
 'trend_vortex_ind_diff',
 'trend_trix',
 'trend_mass_index',
 'trend_dpo',
 'trend_kst',
 'trend_kst_sig',
 'trend_kst_diff',
 'tr

In [105]:
training_data = final_data.fillna(0)

In [106]:
# Define the target variable and features
target = 'ret'
features = [col for col in training_data.columns if col != target and col != 'ret']

# Create X and y arrays
X = training_data[features]
y = training_data[target]

# Check for values too big for float32 and replace with 0s
max_value = np.finfo(np.float32).max
cols_with_big_values = []
for col in features:
    if data[col].max() > max_value:
        cols_with_big_values.append(col)
        num_big_values = (data[col] > max_value).sum()
        print(f'{num_big_values} values in {col} are too big for float32')
        data.loc[data[col] > max_value, col] = 0

# Check for values too big for float32 again after replacing with 0s
for col in cols_with_big_values:
    num_big_values = (data[col] > max_value).sum()
    print(f'{num_big_values} values in {col} are still too big for float32 after replacing with 0')


In [107]:
# Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(training_data[features], training_data[target], test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False) 

# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=7)
rf.fit(X_train, y_train)

C:\Users\Jake Hanly\anaconda3\envs\mlfinlab\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
# Predict the labels of the validation set
y_val_pred = rf.predict(X_val)

In [ ]:
#training set

# Predict labels for training set
y_train_pred = rf.predict(X_train)

# Calculate the confusion matrix, accuracy, precision, recall, and AUC/ROC score for training set
conf_mat = confusion_matrix(y_train, y_train_pred)
acc = accuracy_score(y_train, y_train_pred)
prec = precision_score(y_train, y_train_pred)
rec = recall_score(y_train, y_train_pred)
auc = roc_auc_score(y_train, y_train_pred)

# Print evaluation metrics
print("Confusion Matrix:\n", conf_mat)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("AUC/ROC Score:", auc)


In [ ]:
# Calculate the confusion matrix, accuracy, precision, recall, and AUC/ROC score for validation
conf_mat = confusion_matrix(y_val, y_val_pred)
acc = accuracy_score(y_val, y_val_pred)
prec = precision_score(y_val, y_val_pred)
rec = recall_score(y_val, y_val_pred)
auc = roc_auc_score(y_val, y_val_pred)

# Print the results
print(f'Confusion Matrix:\n{conf_mat}\n')
print(f'Accuracy: {acc:.4f}')
print(f'Precision: {prec:.4f}')
print(f'Recall: {rec:.4f}')
print(f'AUC/ROC: {auc:.4f}')

In [ ]:
# Plot the ROC curve for validation
fpr, tpr, _ = roc_curve(y_val, y_val_pred)
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve, Validation')
plt.show()

In [ ]:
# Feature Importance
title = 'Feature Importance:'
figsize = (15, 5)

feat_imp = pd.DataFrame({'Importance':rf.feature_importances_})    
feat_imp['feature'] = X_train.columns
feat_imp.sort_values(by='Importance', ascending=False, inplace=True)
feat_imp = feat_imp

feat_imp.sort_values(by='Importance', inplace=True)
feat_imp = feat_imp.set_index('feature', drop=True)
feat_imp.plot.barh(title=title, figsize=figsize)
plt.xlabel('Feature Importance Score')
plt.show()

feat_imp.tail(10)

In [ ]:
# Predict the labels of the test set
y_test_pred = rf.predict(X_test)

# Calculate the confusion matrix, accuracy, precision, recall, and AUC/ROC score for test set
conf_mat_test = confusion_matrix(y_test, y_test_pred)
acc_test = accuracy_score(y_test, y_test_pred)
prec_test = precision_score(y_test, y_test_pred)
rec_test = recall_score(y_test, y_test_pred)
auc_test = roc_auc_score(y_test, y_test_pred)

# Print the results for test set
print(f'Test Set Results:')
print(f'Confusion Matrix:\n{conf_mat_test}\n')
print(f'Accuracy: {acc_test:.4f}')
print(f'Precision: {prec_test:.4f}')
print(f'Recall: {rec_test:.4f}')
print(f'AUC/ROC: {auc_test:.4f}')

In [ ]:
# Plot the ROC curve for test set
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_pred)
plt.plot(fpr_test, tpr_test)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Test Set')
plt.show()

In [51]:
# Predict the labels and probabilities of the test set
y_test_pred = rf.predict(X_test)
y_test_pred_proba = rf.predict_proba(X_test)[:, 1]

# Convert the test set predictions and predicted probabilities to a dataframe with date index
test_predictions = pd.DataFrame({'predicted_bin': y_test_pred, 'predicted_proba': y_test_pred_proba}, index=X_test.index)

In [52]:

# Sort the DataFrame by the index to get oldest to newest date order
test_predictions = test_predictions.sort_index()

# Print the results
print(test_predictions)


            predicted_bin  predicted_proba
Date                                      
2022-02-11              1             0.58
2022-02-14              1             0.56
2022-02-15              1             0.53
2022-02-16              1             0.57
2022-02-17              1             0.60
...                   ...              ...
2023-03-06              1             0.52
2023-03-07              1             0.63
2023-03-08              1             0.53
2023-03-09              1             0.53
2023-03-10              1             0.62

[269 rows x 2 columns]
